In [2]:
import numpy as np
import pandas as pd
import scipy.optimize as opt
from __future__ import division

# Logistic Regression

In [3]:
urlTrain = "LogisticData.csv"
df = pd.read_csv(urlTrain)

In [4]:
column = df.shape[1]
X = df.iloc[:,0:column-1]
Y = df.iloc[:,column-1:column]
G=pd.DataFrame()
#groundTruth = df.iloc[:,column-1:column]
G['Y'] = df.iloc[:,column-1:column]
groundTruth = G['Y']

In [5]:
X=pd.DataFrame((X-X.mean())/X.std())
X.insert(0, 'Ones', 1)

In [6]:
X.head()
Y.head()
groundTruth.head()

0    0
1    0
2    0
3    1
4    1
Name: Y, dtype: int64

In [7]:
X = np.matrix(X)  
Y = np.matrix(Y)
m, n = np.shape(X)
print(m , n)

100 3


Define a sigmoid function and return the value tht has been calculated for z

In [8]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

Define the cost function for Logistic Regression. Remember to calculate the sigmoid values as well.

In [9]:
def costFunction(beta, X, Y):
    tempbeta = np.matrix(beta)
    result= np.sum(np.multiply(-Y, np.log(sigmoid(X * tempbeta.T)))-np.multiply((1 - Y), np.log(1 - sigmoid(X * tempbeta.T))))/len(X)
    return result

Define a gradient function that takes in beta, X and Y as parameters and returns the best betas and cost.

In [10]:
def gradientDescent(X, Y, beta, alpha, iters):
    '''
    Compute the gradient descent function.
    Return beta and the cost array.
    '''
    cost = np.zeros(iters)
    m, n = np.shape(X)
    #beta = np.matrix(beta)
    i=0
    for i in range(iters):
        Err=sigmoid(np.dot(X, beta)) - Y
        #print beta, i
        j=0
        for j in range(n):
            tempBeta=beta[j]
            tempBeta=tempBeta-((alpha/m ) * np.sum(np.dot(Err, X[:,j])))
            beta[j]=tempBeta
        cost[i]=costFunction(beta,X,Y)
        
    #print beta[0], i
    return beta, cost

In [11]:
#please try different values to see the results, but alpha=0.01 and iters=10000 are suggested.
beta = np.zeros(n)
alpha = 0.01
iters = 10000
resultDescent = gradientDescent(X, Y, beta, alpha, iters)

In [12]:
print("Beta From Gradient Descent : " ,resultDescent[0])

Beta From Gradient Descent :  [ 4.05465108e-01 -8.21565038e-18 -2.04281037e-17]


In [13]:
def gradient(beta, X, Y):
    '''
    This function returns the gradient calucated.
    '''
    #for i in range(parameters):
    #####
    #grad[i] =
    m, n = np.shape(X)
    grad = np.zeros(n)
    Err = sigmoid(np.dot(X, beta)) - Y
    for j in range(n):
        term = np.multiply(Err, X[:,j])
        grad[j] = np.sum(term) / m
    return grad

Optimize the parameters given functions to compute the cost and the gradients. We can use SciPy's optimization to do the same thing. Define a variable result and complete the functions by adding the right parameters.

In [14]:
#the optimised betas are stored in the first index of the result variable
beta = np.zeros(n)
result = opt.fmin_tnc(func=costFunction , x0=beta , fprime =gradient , args=(X,Y) )

In [15]:
print(result)

(array([1.54810212, 4.33189095, 3.86604543]), 83, 1)


In [16]:
def predict(bestBeta, X): 
    '''
    This function returns a list of predictions calculated from the sigmoid using the best beta.
    '''
    probability = sigmoid(np.dot(X, bestBeta))
    p, q= probability.shape
    #print p , q
    Res = []
    for x in range(q):
        if probability[0,x]>=0.5:
            Res.append(1)
        else:
            Res.append(0)
    return Res, probability

In [17]:
bestBeta = result[0] 
print("Best Beta : ",bestBeta)
print("Cost for Best Beta : ",costFunction(bestBeta,X,Y))

Best Beta :  [1.54810212 4.33189095 3.86604543]
Cost for Best Beta :  0.20645819114071567


In [18]:
def confusionMatrix(prediction, ground1):
    '''
    Return the computed confusion matrix.
    '''
    return pd.crosstab( ground1,prediction, rownames=['True'], colnames=['Predicted'], margins=False)

In [19]:
def getAccuracy(groundTruth, predictions):
    correct = 0
    for i in range(len(groundTruth)):
        if groundTruth[i] == predictions[i]:
            correct += 1
    return (correct/float(len(groundTruth))) * 100.0

Call the confusionMatrix function and print the confusion matrix as well as the accuracy of the model.

In [20]:
#The final outputs that we need for this portion of the lab are conf and acc. Copy conf and acc in a .txt file.
#Please write a SHORT report and explain these results. Include the explanations for both logistic and linear regression
#in the same PDF file. 

groundTruth1=groundTruth.as_matrix()
#print groundTruth1
predictions = predict(bestBeta, X)
predictionVal=predictions[0]
ProbVal=predictions[1]
predictionVal=np.array(predictionVal)
y_actu = pd.Series(groundTruth)
y_pred = pd.Series(predictionVal)
conf = confusionMatrix(y_pred,y_actu)
#print y_actu
print(conf)
acc = getAccuracy(groundTruth1, predictionVal)
print(acc)
print('accuracy = ',(acc))

Predicted   0   1
True             
0          35   5
1           5  55
90.0
accuracy =  90.0
